In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('files\\dataset_ksg.csv')

len(df)

446976

In [6]:
keys = df['obj_key'].unique()

In [21]:
key_df = df[df['obj_key'] == keys[0]]
main_keys = ['1', '1.4.2', '3', '3.1', '3.3', '3.8', '3.11', '3.13', '3.15', '4', '4.1.1', '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', '4.10', '4.11', '4.12', '5', '5.11', '7.1', '7.1.8', '7.4', '8']
key_df[key_df['Кодзадачи'].isin(main_keys)]

,№ п/п,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,ДатаначалаБП0,ДатаокончанияБП0,Статуспоэкспертизе,Экспертиза,date_report
0,81,022-0527,1,0.0,2022-01-14,2023-03-30,NaN,NaN,NaN,NaN,2023.01.17
258,81,022-0527,1.4.2,0.0,2023-01-09,2023-03-30,NaN,NaN,NaN,NaN,2023.01.17
2247,81,022-0527,3,0.0,2023-01-17,2024-02-01,NaN,NaN,NaN,NaN,2023.01.17
2415,81,022-0527,3.1,0.0,2023-01-17,2023-05-22,2022-02-25,2022-05-01,NaN,NaN,2023.01.17
2549,81,022-0527,3.11,0.0,2023-03-27,2023-08-24,NaN,2022-07-15,NaN,NaN,2023.01.17
...,...,...,...,...,...,...,...,...,...,...,...
440279,84,022-0527,5,0.0,2024-06-08,2024-09-24,2024-04-05,2024-07-20,NaN,NaN,2023.05.29
440621,84,022-0527,5.11,0.0,2024-08-04,2024-08-08,2024-06-01,2024-06-05,NaN,NaN,2023.05.29
444293,84,022-0527,7.1,0.0,2024-06-12,2024-07-08,2024-05-29,2024-06-24,NaN,NaN,2023.05.29
444970,84,022-0527,7.1.8,0.0,2024-07-04,2024-07-08,2024-06-20,2024-06-24,NaN,NaN,2023.05.29
